In [364]:
! pip install pandas
! pip install openpyxl
import pandas as pd

In [365]:
data = pd.ExcelFile("Rearranged_Sheet.xlsx")
sheet_names = data.sheet_names
sheet_names

['PROJECT',
 'OUTPUT 1',
 'OUTPUT 2',
 'OUTPUT 3',
 'OUTPUT 4',
 'OUTPUT 5',
 'OUTPUT 6',
 'OUTPUT 7',
 'INDICATOR 1',
 'INDICATOR 2',
 'INDICATOR 3',
 'INDICATOR 4',
 'INDICATOR 5',
 'INDICATOR 6',
 'INDICATOR 7']

In [366]:
project = data.parse('PROJECT', dtype={'DONOR CODE': object}).copy()
project

,PROJECT_TITLE,PROJECT_ID,PROJECT_DESCRIPTION,TOTAL_REQUIRED_RESOURCE,TOTAL_AVAILABLE_RESOURCE,RESOURCE_TYPE,DONOR_NAME,DONOR_CODE,AMOUNT,COMMENT,CLEARANCE_ROLE,NAME,APPROVAL_DATE,BOARD_DATE
0,Engagement facility West and Central Africa an...,00092084 / 00117764 / 00126438,NaN,442 398.5,442 398.5,CORE,NaN,NaN,0.0,Expected contribution of 86240 USD early 2025 ...,Project Manager,David Larue,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,DONOR,UNFPA,12445.0,111240.0,NaN,Director/Deputy Director for the Practice area,Henri Dommel,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,DONOR,LUXEMBOURG,155.0,305696.5,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,DONOR,UNDP,12.0,25462.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,IN KIND,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Host Government or stakeholder,NaN,NaN,NaN


In [367]:
project_columns = project.columns
project_columns

Index(['PROJECT_TITLE', 'PROJECT_ID', 'PROJECT_DESCRIPTION',
       'TOTAL_REQUIRED_RESOURCE', 'TOTAL_AVAILABLE_RESOURCE', 'RESOURCE_TYPE',
       'DONOR_NAME', 'DONOR_CODE', 'AMOUNT', 'COMMENT', 'CLEARANCE_ROLE',
       'NAME', 'APPROVAL_DATE', 'BOARD_DATE'],
      dtype='object')

In [368]:
def filterList(item_list, index_list):
    new_list = []
    for index in index_list:
        new_list.append(item_list[index])
        
    return new_list

project_table_column = filterList(project_columns, [0,1,2,3,4,9,12])
project_table_column
       

['PROJECT_TITLE',
 'PROJECT_ID',
 'PROJECT_DESCRIPTION',
 'TOTAL_REQUIRED_RESOURCE',
 'TOTAL_AVAILABLE_RESOURCE',
 'COMMENT',
 'APPROVAL_DATE']

In [369]:
project_table = project[project_table_column]

project_table = project_table.dropna(subset=['PROJECT_ID'])
this_project_id = project_table.iloc[0]['PROJECT_ID']
project_table

,PROJECT_TITLE,PROJECT_ID,PROJECT_DESCRIPTION,TOTAL_REQUIRED_RESOURCE,TOTAL_AVAILABLE_RESOURCE,COMMENT,APPROVAL_DATE
0,Engagement facility West and Central Africa an...,00092084 / 00117764 / 00126438,NaN,442 398.5,442 398.5,Expected contribution of 86240 USD early 2025 ...,NaN


In [370]:
donor_table = project[['DONOR_NAME', 'DONOR_CODE']]
donor_table = donor_table.dropna()
donor_table

,DONOR_NAME,DONOR_CODE
1,UNFPA,12445.0
2,LUXEMBOURG,155.0
3,UNDP,12.0


In [371]:
project_resources_table = project[filterList(project_columns, [5,7,8])]
project_resources_table = project_resources_table.dropna(subset=[project_columns[5]])
project_resources_table['PROJECT_ID'] = this_project_id
project_resources_table

,RESOURCE_TYPE,DONOR_CODE,AMOUNT,PROJECT_ID
0,CORE,NaN,0.0,00092084 / 00117764 / 00126438
1,DONOR,12445.0,111240.0,00092084 / 00117764 / 00126438
2,DONOR,155.0,305696.5,00092084 / 00117764 / 00126438
3,DONOR,12.0,25462.0,00092084 / 00117764 / 00126438
4,IN KIND,NaN,0.0,00092084 / 00117764 / 00126438


In [372]:
project_post_title_table = project[filterList(project_columns, [10, 11])]
project_post_title_table = project_post_title_table.dropna()
project_post_title_table = project_post_title_table.rename(columns={'CLEARANCE_ROLE': 'POST_TITLE'})
project_post_title_table['PROJECT_ID'] = this_project_id
project_post_title_table

,POST_TITLE,NAME,PROJECT_ID
0,Project Manager,David Larue,00092084 / 00117764 / 00126438
1,Director/Deputy Director for the Practice area,Henri Dommel,00092084 / 00117764 / 00126438


In [373]:
project_approval_table = project[['CLEARANCE_ROLE', 'NAME', 'APPROVAL_DATE']]
project_approval_table = project_approval_table.dropna(subset=['CLEARANCE_ROLE'])
project_approval_table['PROJECT_ID'] = this_project_id
project_approval_table

,CLEARANCE_ROLE,NAME,APPROVAL_DATE,PROJECT_ID
0,Project Manager,David Larue,NaN,00092084 / 00117764 / 00126438
1,Director/Deputy Director for the Practice area,Henri Dommel,NaN,00092084 / 00117764 / 00126438
5,Host Government or stakeholder,NaN,NaN,00092084 / 00117764 / 00126438


In [374]:
count = 2
activity = data.parse(f'OUTPUT {count}', header=2)
activity
#new_activity = pd.concat([activity, activity2], ignore_index=True)


,PLANNED_ACTIVITY,COUNTRY,Q1,Q2,Q3,Q4,RESPONSIBLE_PARTY,DONOR,DONOR_CODE,BUDGET_DESCRIPTION,ANNUAL_BUDGET,AMOUNT_FUNDED,AMOUNT_UNFUNDED,GMS,Q1_EXPENDITURE,Q2_EXPENDITURE,Q3_EXPENDITURE,Q4_EXPENDITURE,PROGRESS
0,Activity 2.1 Support for the structuring of yo...,Burkina Faso,NaN,4000,NaN,NaN,UNCDF,UNFPA,12445,72100- Contractual Services,4000,4000,0,8,NaN,NaN,NaN,NaN,NaN
1,Activty 2.2:Organizing a awarness day on finan...,Burkina Faso,NaN,5000,NaN,NaN,UNCDF,UNFPA,12445,72510- Publications,5000,5000,0,8,NaN,NaN,NaN,NaN,NaN


In [375]:
outcome = data.parse('OUTPUT 2').iloc[0:1]
outcome = outcome.dropna(axis=1, how='all')
outcome['PROJECT_ID'] = this_project_id
outcome

,PROJECT_OUTCOME,PROJECT_OUTPUT,PROJECT_ID
0,Outcome 1 : UNFPA UN2UN agreement in Burkina Faso,Output 2: The structuring of youth and adolesc...,00092084 / 00117764 / 00126438


In [376]:
activity['PROJECT_OUTPUT'] = outcome.iloc[0]['PROJECT_OUTPUT']
col = activity.columns.to_list()
activity = activity[col[-1:] + col[:-1]]
activity_table = activity.drop(columns=['DONOR', 'BUDGET_DESCRIPTION', 'COUNTRY'])
activity

,PROJECT_OUTPUT,PLANNED_ACTIVITY,COUNTRY,Q1,Q2,Q3,Q4,RESPONSIBLE_PARTY,DONOR,DONOR_CODE,BUDGET_DESCRIPTION,ANNUAL_BUDGET,AMOUNT_FUNDED,AMOUNT_UNFUNDED,GMS,Q1_EXPENDITURE,Q2_EXPENDITURE,Q3_EXPENDITURE,Q4_EXPENDITURE,PROGRESS
0,Output 2: The structuring of youth and adolesc...,Activity 2.1 Support for the structuring of yo...,Burkina Faso,NaN,4000,NaN,NaN,UNCDF,UNFPA,12445,72100- Contractual Services,4000,4000,0,8,NaN,NaN,NaN,NaN,NaN
1,Output 2: The structuring of youth and adolesc...,Activty 2.2:Organizing a awarness day on finan...,Burkina Faso,NaN,5000,NaN,NaN,UNCDF,UNFPA,12445,72510- Publications,5000,5000,0,8,NaN,NaN,NaN,NaN,NaN


In [377]:
indicator = data.parse('INDICATOR 2')
indicator['PROJECT_OUTPUT'] = outcome.iloc[0]['PROJECT_OUTPUT']
i_col = indicator.columns.to_list()
indicator = indicator[i_col[-1:] + i_col[:-1]]
indicator.columns.to_list()

['PROJECT_OUTPUT',
 'INDICATOR',
 'DISAGGREGATION',
 'BASELINES',
 'YEAR',
 'ANNUAL_TARGET',
 'UNIT',
 'MID_YEAR_ACTUALS',
 'END_YEAR_ACTUALS',
 'DATA_SOURCE']

In [378]:
budget_code_table = pd.DataFrame(columns=['CODE', 'DESCRIPTION'])
activity_budget = pd.DataFrame(columns=['BUDGET_CODE', 'PROJECT_ACTIVITY'])
budget = activity['BUDGET_DESCRIPTION']
for row in budget:

    for str in row.split(','):
        code =[str.split('-')[0].strip()]
        bc = pd.DataFrame({
            'CODE':code,
            'DESCRIPTION': str.split('-')[1].strip()
        })
        
        budget_code_table = pd.concat([budget_code_table, bc] ,ignore_index=True )

        
budget_code_table

,CODE,DESCRIPTION
0,72100,Contractual Services
1,72510,Publications


In [380]:
count = 1
# The tables to be created with each sheets
budget_code_table = pd.DataFrame(columns=['CODE', 'DESCRIPTION'])
activity_budget_table = pd.DataFrame(columns=['PLANNED_ACTIVITY', 'BUDGET_CODE'])
output_table = pd.DataFrame(columns=['PROJECT_ID', 'PROJECT_OUTCOME', 'PROJECT_OUTPUT'])
indicator_table = pd.DataFrame(columns=[
 'PROJECT_OUTPUT',
 'INDICATOR',
 'DISAGGREGATION',
 'BASELINES',
 'YEAR',
 'ANNUAL_TARGET',
 'UNIT',
 'MID_YEAR_ACTUALS',
 'END_YEAR_ACTUALS',
 'DATA_SOURCE'])


while True:
    try:
        # sort out the project activities
        act = data.parse(f'OUTPUT {count}', header=2)
       
        for index, row in act.iterrows():
            # separate budget codes and activities
            bud = row['BUDGET_DESCRIPTION']
            
            if not pd.isnull(bud):
                for str in bud.split(','):
                    b = pd.DataFrame({
                        'CODE': str.split('-')[0].strip().upper(),
                        'DESCRIPTION': str.split('-')[1].strip().upper()
                    })
                    print(b)
                    budget_code_table = pd.concat([budget_code_table, b], ignore_index=True)
                    

        # adding to the project outcome and outputs
        oc = data.parse(f'OUTPUT {count}').iloc[0:1]
        oc = oc.dropna(axis=1, how='all')
        oc['PROJECT_ID'] = this_project_id
        oc = oc[output_table.columns.to_list()]
        output_table = pd.concat([output_table, oc], ignore_index=True)
        
    except ValueError:
        break
    
    try:
        # sort out the project indicators
        ind = data.parse(f'INDICATOR {count}')
        ind['PROJECT_OUTPUT'] = oc.iloc[0]['PROJECT_OUTPUT']
        indicator_table = pd.concat([indicator_table, ind], ignore_index=True)
        indicator_table.drop_duplicates()
    except ValueError:
        pass
    
    print(count)
    
    count += 1
    
indicator_table
    

,PROJECT_OUTPUT,INDICATOR,DISAGGREGATION,BASELINES,YEAR,ANNUAL_TARGET,UNIT,MID_YEAR_ACTUALS,END_YEAR_ACTUALS,DATA_SOURCE
